KnockoffDB
---

This class is responsible for building the tables and inserting the data into the database. It accomplishes this using a **KnockoffDatabaseService** provided to its \_\_init\_\_ to interact with the database for getting table definitions and uploading knockoff data. The **DefaultDatabaseService** is an implmentation of the **KnockoffDatabaseService** that can be used out of the box to interact with any sqlalchemy compatible database.

There are various patterns that can leveraged for configuring a **KnockoffDB** instance with a **KnockoffDatabaseService** and **KnockoffTable** instances, but in this example we'll use the classes and objects directly from the sdk. The other patterns (e.g. using a **Blueprint**) will be show cased in other examples, but are to support dependency injection and composability for pytest fixtures and for the CLI.


## Example

Preliminary knowledge of the [KnockoffTable](KnockoffTable.ipynb) is recommended for this example. For this example we'll create a basic data model with the following tables: Product, Location, Transaction.

* [Product Table](#Product-Table)
* [Location Table](#Location-Table)
* [Transaction Table](#Transaction-Table)
* [KnockoffDB](#KnockoffDB-ex)


### Product Table

For this example we want our product table to satisfy a hierachy to resemble more realistic data. To do so, we'll first need to generate a hiearchy DataFrame.

In [1]:
import pandas as pd

shoes_categories = [
    "Lifestyle",
    "Running",
    "Basketball",
    "Jordan",
    "Training & Gym",
    "Soccer",
    "Golf",
    "Cross Country",
    "Skateboarding",
    "Tennis",
    "Baseball",
    "Sandals & Slides"
]

apparel_categories = [
    "Tops & T-Shirts",
    "Shorts",
    "Hoodies & Sweatshirts",
    "Pants & Tights",
    "Matching Sets",
    "Jackets & Vests",
    "Swimwear",
    "Polos",
    "Yoga",
    "Socks",
    "Underwear",
    "Big & Tall",
    "Sustainable Materials"
]


def cartesian_product(values, names):
    return pd.DataFrame(
        index=pd.MultiIndex.from_product(
            values, 
            names=names
        )
    ).reset_index()

shoes = cartesian_product(
    [["Shoes"], shoes_categories], 
    names=["division", "category"]
)

apparel = cartesian_product(
    [["Apparel"], apparel_categories], 
    names=["division", "category"]
)

hierarchy = pd.concat([shoes, apparel])
display(hierarchy)


,division,category
0,Shoes,Lifestyle
1,Shoes,Running
2,Shoes,Basketball
3,Shoes,Jordan
4,Shoes,Training & Gym
5,Shoes,Soccer
6,Shoes,Golf
7,Shoes,Cross Country
8,Shoes,Skateboarding
9,Shoes,Tennis


We'll also want valid size values based on the division so here we'll define a **ColumnFactory** with a dependency on **division**.

In [2]:
from knockoff.sdk.factory.column import ColumnFactory, FakerFactory, ChoiceFactory

def size_factory(division):
    return {
        "Apparel": ChoiceFactory(["XS", "S", "M", "L", "XL", "XXL"]),
        "Shoes": FakerFactory("pyint", min_value=4, max_value=16),
    }[division]()

Now we can define our Product KnockoffTable.

In [3]:
from knockoff.sdk.table import KnockoffTable
from knockoff.sdk.factory.collections import KnockoffDataFrameFactory
from knockoff.sdk.constraints import KnockoffUniqueConstraint

product_table = KnockoffTable(
    "product",
    columns=[
        "division",
        "category",
        "gender",
        "color",
        "size",
        "sku",
        "price"
    ],
    factories=[
        KnockoffDataFrameFactory(hierarchy),
        ColumnFactory("gender", ChoiceFactory(["Mens", "Womens"])),
        ColumnFactory("color", FakerFactory("color_name")),
        ColumnFactory("size", size_factory, depends_on=["division"]),
        ColumnFactory("sku", FakerFactory("numerify", text="%#########")),
        ColumnFactory("price", FakerFactory("pyfloat", right_digits=2, min_value=5, max_value=300)),
    ],
    constraints=[KnockoffUniqueConstraint(["sku"])],
    size=50
)

### Location Table

In [4]:
location_table = KnockoffTable(
    "location",
    columns=[
        "location_id",
        "address",
    ],
    factories=[
        ColumnFactory("location_id", FakerFactory("numerify", text="%###")),
        ColumnFactory("address", FakerFactory("address")),
    ],
    constraints=[KnockoffUniqueConstraint(["location_id"])],
    size=10
)

### Transaction Table

For the transaction table we'll need to use the KnockoffTableFactory to model foreign key relationships.

In [5]:
import numpy as np
from knockoff.sdk.factory.collections import KnockoffTableFactory

# create a demand function
def sales_func():
    return max(np.random.poisson(1), 1)

# create a revenue function based on price and units
def revenue_func(price, units):
    return price*float(units)


transaction_table = KnockoffTable(
    "transaction",
    columns=[
        "location_id",
        "sku",
        "units",
        "revenue",
        "date"
    ],
    factories=[
        KnockoffTableFactory(product_table, columns=["sku", "price"]),
        KnockoffTableFactory(location_table, columns=["location_id"]),
        ColumnFactory("units", sales_func),
        ColumnFactory("revenue", revenue_func, depends_on=["price", "units"]),
        ColumnFactory("date", FakerFactory("date_between", start_date="-2y", end_date="today")),
    ],
    size=200
)


### <a name="KnockoffDB-ex"></a>KnockoffDB

Now that we've defined our tables, we can provide them to the KnockoffDB.

In [6]:
from sqlalchemy import create_engine
from knockoff.sdk.db import KnockoffDB, DefaultDatabaseService

# The database service is actually only required
# if we want to autoload the table definitions or
# if we want to insert the data. For this example
# we'll simply build the DataFrames that would be
# inserted.
database_service = None 

# Below is an example of configuring the DefaultDatabaseService
# url = .. # connection url to database
# database_service = DefaultDatabaseService(create_engine(url))


# Instantiate KnockoffDB instance
knockoff_db = KnockoffDB(database_service)

# Add tables with dependencies
knockoff_db.add(transaction_table, 
                depends_on=["product", "location"])
knockoff_db.add(product_table) # these can be added in any order
knockoff_db.add(location_table)

dfs = knockoff_db.build() # if we had provided an actual database service, 
                          # calling insert() would load these into the database

In [7]:
display(dfs.keys())

display(dfs['location'].head())
display(dfs['product'].head())
display(dfs['transaction'].head())

dict_keys(['location', 'product', 'transaction'])

,location_id,address
0,3985,"6973 Wong Radial Suite 361\nNorth Brandon, CA ..."
1,3125,"8242 Jacob Crossing\nSouth Andre, MI 71867"
2,9949,"274 Ryan Forge Suite 432\nLake Rodney, AL 73237"
3,4546,"239 Michaela Spring Suite 435\nJonathonberg, K..."
4,5596,USS Montgomery\nFPO AP 36767


,division,category,gender,color,size,sku,price
0,Apparel,Socks,Womens,DarkViolet,XL,2778986833,71.44
1,Apparel,Big & Tall,Womens,Lavender,M,6425191615,29.36
2,Shoes,Lifestyle,Mens,Yellow,5,3418806930,108.36
3,Apparel,Big & Tall,Mens,PapayaWhip,M,9215866886,122.71
4,Shoes,Sandals & Slides,Womens,NavajoWhite,10,2546345047,131.96


,location_id,sku,units,revenue,date
0,7723,4154009234,1,299.32,2021-04-21
1,9949,6584917951,1,280.11,2021-01-27
2,4396,7777086793,1,267.46,2020-07-25
3,3125,8116513548,1,198.37,2019-10-27
4,6537,4339201699,2,361.98,2021-03-10
